# Disclaimer & Copyright

Copyright 2024 Forusone : shins777@gmail.com

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at

https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

# Lab for basic text operation with Gemini and understanding Gemini fundamental.


## Install python packages
* Vertex AI SDK for Python
  * https://cloud.google.com/python/docs/reference/aiplatform/latest

In [1]:
%pip install --upgrade --quiet google-cloud-aiplatform

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 31.4 MB/s eta 0:00:00


## Authentication to access to the GCP & Google drive

In [2]:
# To use markdown for output data from LLM
from IPython.display import display, Markdown

# Use OAuth to access the GCP environment.
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()


## Set the environment on GCP Project

In [4]:
# Set the environment on GCP Project
PROJECT_ID="ai-hangsik"
REGION="asia-northeast3"
MODEL_NAME="gemini-1.5-flash"

## Vertex AI initialization


In [5]:
import vertexai

from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    HarmBlockThreshold,
    HarmCategory,
    Part,
)

# Initialize the current vertex AI execution environment.
vertexai.init(project=PROJECT_ID, location=REGION) # https://cloud.google.com/python/docs/reference/aiplatform/latest#initialization

system_instructions = [
 "Respond in Korean",
 "Respond in structured ways"
]

# Access to the generative model.
# https://cloud.google.com/vertex-ai/generative-ai/docs/reference/python/latest/vertexai.generative_models.GenerativeModel
model = GenerativeModel(MODEL_NAME,
                        system_instruction=system_instructions)

## Function to get the response

In [7]:
def generate(query:str, stream:bool):
    """
    Generate a response from the model.
    query :query to be sent to the model
    Returns:The generated response.
    """

    # Request body : https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference#request
    # parameter : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/send-multimodal-prompts#set_model_parameters
    generation_config = GenerationConfig(
        temperature=0.9,
        top_p=1.0,
        top_k=32,
        candidate_count=1,
        max_output_tokens=8192,
    )

    contents = [query]

    # Counts tokens
    print(f"Token Count : {model.count_tokens(contents)}")

    # Configure satey setting : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/configure-safety-attributes
    # Refer to the link to remove : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/configure-safety-attributes#how_to_remove_automated_response_blocking_for_select_safety_attributes
    safety_settings = {
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    }

    responses = model.generate_content(
        contents,
        generation_config=generation_config,
        safety_settings=safety_settings,
        stream=stream,
    )

    return responses



## Run example

In [8]:

from time import perf_counter

t1_start = perf_counter()

prompt = """
          You are a Generative AI specialist to help people understand the concept of AI.
          Explain what's the pros and cons of using AI in the following instructions.

          1. Define what's the Generative AI.
          2. What are differences between typical machine learning and Generative AI.
          3. What are the pros and cons of using Generative AI.
          4. Explain use cases of Generative AI in the manufacturing industry.
"""
stream = False

responses = generate(prompt, stream)

if stream:
  for response in responses:
    print(response.text, end="")
else:
  # Print the model response
  print(f'\n Usage metadata:\n{responses.to_dict().get("usage_metadata")}')
  print(f"\n Finish reason:\n{responses.candidates[0].finish_reason}")
  print(f"\n Safety settings:\n{responses.candidates[0].safety_ratings}")

  display(Markdown(responses.text))

t1_end  = perf_counter()
print(f"Time : {t1_end - t1_start} seconds\n\n")



Token Count : total_tokens: 105
total_billable_characters: 363


 Usage metadata:
{'prompt_token_count': 105, 'candidates_token_count': 775, 'total_token_count': 880}

 Finish reason:
1

 Safety settings:
[category: HARM_CATEGORY_HATE_SPEECH
probability: NEGLIGIBLE
probability_score: 0.0206298828
severity: HARM_SEVERITY_NEGLIGIBLE
severity_score: 0.0903320312
, category: HARM_CATEGORY_DANGEROUS_CONTENT
probability: NEGLIGIBLE
probability_score: 0.0219726562
severity: HARM_SEVERITY_NEGLIGIBLE
severity_score: 0.103515625
, category: HARM_CATEGORY_HARASSMENT
probability: NEGLIGIBLE
probability_score: 0.0913085938
severity: HARM_SEVERITY_NEGLIGIBLE
severity_score: 0.0805664062
, category: HARM_CATEGORY_SEXUALLY_EXPLICIT
probability: NEGLIGIBLE
probability_score: 0.0275878906
severity: HARM_SEVERITY_NEGLIGIBLE
severity_score: 0.056640625
]


## 생성형 AI에 대한 안내: 장점과 단점, 그리고 제조업 적용 사례

**1. 생성형 AI란 무엇인가요?**

생성형 AI는 새로운 콘텐츠를 만들어내는 인공지능의 한 유형입니다. 텍스트, 이미지, 오디오, 비디오, 코드 등 다양한 형태의 콘텐츠를 생성할 수 있습니다. 기존 데이터를 학습하여 새로운 콘텐츠를 생성하는 것이 특징입니다.

**2. 일반적인 머신러닝과 생성형 AI의 차이점은 무엇인가요?**

* **일반적인 머신러닝:**  주어진 데이터를 기반으로 미래를 예측하거나 분류하는 데 사용됩니다. 예를 들어, 이메일 스팸 필터링이나 이미지 분류가 있습니다.
* **생성형 AI:**  새로운 콘텐츠를 생성합니다. 예를 들어, 텍스트 생성, 이미지 생성, 음악 작곡 등이 있습니다.

**3. 생성형 AI를 사용하는 장점과 단점은 무엇인가요?**

**장점:**

* **창의성 향상:**  새로운 아이디어를 탐구하고, 기존 방식에서 벗어난 혁신적인 솔루션을 제공할 수 있습니다.
* **생산성 증대:**  반복적인 작업을 자동화하여 시간을 절약하고 생산성을 높일 수 있습니다.
* **개인 맞춤형 콘텐츠 제공:**  개인의 취향에 맞는 맞춤형 콘텐츠를 생성하여 고객 만족도를 높일 수 있습니다.
* **데이터 부족 문제 해결:**  데이터가 부족한 상황에서도 새로운 데이터를 생성하여 AI 모델 학습을 가능하게 합니다.

**단점:**

* **데이터 편향 문제:**  학습 데이터에 편향이 존재할 경우, 생성된 콘텐츠에도 편향이 반영될 수 있습니다.
* **저작권 및 윤리적 문제:**  생성된 콘텐츠의 저작권 및 윤리적 문제에 대한 명확한 기준이 부족합니다.
* **오류 및 부정확성:**  생성된 콘텐츠가 항상 정확하거나 완벽하지 않을 수 있습니다.
* **데이터 보안 문제:**  학습 데이터 및 생성된 콘텐츠의 보안 문제에 대한 우려가 존재합니다.

**4. 제조업에서 생성형 AI를 활용하는 사례는 무엇인가요?**

* **제품 설계 및 개발:**  새로운 제품 디자인을 생성하고, 제품 성능을 시뮬레이션하여 개발 시간을 단축할 수 있습니다.
* **생산 계획 및 예측:**  수요 예측 및 생산 계획을 최적화하여 생산 효율성을 높일 수 있습니다.
* **품질 관리:**  생산 과정에서 발생하는 불량품을 자동으로 감지하고, 품질 관리 시스템을 개선할 수 있습니다.
* **로봇 제어 및 자동화:**  생성형 AI를 활용하여 로봇의 동작을 계획하고, 자동화 시스템을 구축할 수 있습니다.

**결론:**

생성형 AI는 제조업을 포함한 다양한 분야에서 큰 잠재력을 가지고 있습니다. 하지만, 기술의 발전과 함께 윤리적 문제 및 보안 문제를 해결하기 위한 노력이 필요합니다.


Time : 6.410154087999956 seconds


